In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from datetime import datetime

In [51]:
def dividir_columnas(df):
    columnas_transformadas = {}
    columnas_no_transformadas = []
    columnas_total = list(df.columns)
    for col in columnas_total:
        if col in list(correspondencias.keys()):
            columnas_transformadas[col] = correspondencias[col]
        else:
            columnas_no_transformadas.append(col)
    return (columnas_transformadas, columnas_no_transformadas)

def transformar_datos(df_datos, cols_t):
    datos_finales = pd.DataFrame(df_datos[list(cols_t.keys())])
    datos_finales = datos_finales.rename(columns = cols_t)
    
    datos_finales["DMRAGEYR"] = datos_finales["DMRAGEYR"].apply(lambda x: x if isinstance(x, int) and 0 <= x <= 150 else '.')
    mapping_gen = {'Male': 0, 'FEMALE': 1}
    datos_finales["DMRGENDR"] = datos_finales["DMRGENDR"].map(mapping_gen).fillna('.')
    datos_finales["DATAD"] = pd.to_datetime(datos_finales["DATAD"], errors='coerce').apply(lambda x: x.strftime('%d/%m/%Y') if not pd.isnull(x) else '.')
    datos_finales["DATLGTI"] = datos_finales["DATLGTI"].apply(lambda x: x if isinstance(x, int) and 0 <= x <= 180 else '.')
    datos_finales["DATDSI"] = pd.to_datetime(datos_finales["DATDSI"], errors='coerce').apply(lambda x: x.strftime('%d/%m/%Y') if not pd.isnull(x) else '.')
    mapping_out_status = {'Recovered':0,'Deceased':1,'Transferred':2}
    datos_finales["DSXOS"] = datos_finales["DSXOS"].map(mapping_out_status).fillna('.')
    datos_finales["CSXBTPA"] = datos_finales["CSXBTPA"].apply(lambda x: round(x) if isinstance(x, int) and 25 <= x <= 43 else '.')
    datos_finales["CSXCHRA"] = datos_finales["CSXCHRA"].apply(lambda x: x if isinstance(x, int) and 0 <= x <= 288 else '.')
    datos_finales["LBXSGLA"] = datos_finales["LBXSGLA"].apply(lambda x: x if isinstance(x, int) and 33 <= x <= 559 else '.')
    datos_finales["CSXOSTA"] = datos_finales["CSXOSTA"].apply(lambda x: x if isinstance(x, int) and 0 <= x <= 100 else '.')
    datos_finales["CSXSYA"] = datos_finales["CSXSYA"].apply(lambda x: x if isinstance(x, int) and 74 <= x <= 270 else '.')
    datos_finales["CSXDIA"] = datos_finales["CSXDIA"].apply(lambda x: x if isinstance(x, int) and 0 <= x <= 124 else '.')
    datos_finales["CSXCHRHn"] = datos_finales["CSXCHRHn"].apply(lambda x: x if isinstance(x, int) and 0 <= x <= 288 else '.')
    datos_finales["CSXBTPHn"] = datos_finales["CSXBTPHn"].apply(lambda x: round(x,1) if isinstance(x, int) and 25 <= x <= 43 else '.')
    datos_finales["LBXSGLHn"] = datos_finales["LBXSGLHn"].apply(lambda x: x if isinstance(x, int) and 33 <= x <= 559 else '.')
    datos_finales["CSXOSTHn"] = datos_finales["CSXOSTHn"].apply(lambda x: x if isinstance(x, int) and 0 <= x <= 100 else '.')
    datos_finales["CSXSYHn"] = datos_finales["CSXSYHn"].apply(lambda x: x if isinstance(x, int) and 74 <= x <= 270 else '.')
    datos_finales["CSXDIHn"] = datos_finales["CSXDIA"].apply(lambda x: x if isinstance(x, int) and 0 <= x <= 124 else '.')

    return datos_finales

def df_to_sql_insert(df, nombre_tabla):
    res = "INSERT INTO " + nombre_tabla +" ("
    for col in list(df.columns):
        res += col + ", "
    res = res[:-2] + ") VALUES (" + "%s, " * len(df.columns)
    res = res[:-2] + ")"
    return res

def cargar_datos_finales(df):
    df = df.applymap(lambda x: str(x))
    sql_carga_datos_finales = df_to_sql_insert(df, "datos_finales")
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    for i in range(len(df)):
        cursor.execute(sql_carga_datos_finales, list(df.iloc[i]))
        conn.commit()
    sql_PATIENT_IDs = "SELECT PATIENT_ID FROM datos_finales ORDER BY PATIENT_ID DESC"
    cursor.execute(sql_PATIENT_IDs)
    ids = cursor.fetchall()
    id_final = ids[0][0]
    id_inicial = id_final - len(df) + 1
    cursor.close()
    conn.close()
    return id_inicial

def iniciar_carga(name):
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    fecha = str(datetime.now())[:-7]
    sql_carga = "INSERT INTO cargas (fecha) VALUES (%s)"
    cursor.execute(sql_carga, [fecha])
    conn.commit()
    sql_carga2 = "INSERT INTO cargas (nombre_archivo) VALUES (%s)"
    cursor.execute(sql_carga2, [name])
    conn.commit()
    sql_id_carga = "SELECT ID_carga FROM cargas WHERE fecha = %s"
    cursor.execute(sql_id_carga, [fecha])
    id_carga =cursor.fetchone()[0]
    cursor.close()
    conn.close()
    return id_carga

def carga_columnas_transformadas(cols_t, id_carga):
    df_cols_transformadas = pd.DataFrame({"nombre_original": list(cols_t.keys()),
                                      "nombre_final": list(cols_t.values()),
                                     "ID_carga": str(id_carga)})
    sql_carga_cols_t = df_to_sql_insert(df_cols_transformadas, "columnas_transformadas")
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    for i in range(len(df_cols_transformadas)):
        cursor.execute(sql_carga_cols_t, list(df_cols_transformadas.iloc[i]))
        conn.commit()
    cursor.close()
    conn.close()
    
def carga_columnas_no_transformadas(cols_n, id_carga):
    df_cols_no_transformadas = pd.DataFrame({"nombre": cols_n,
                                     "ID_carga": str(id_carga)})
    sql_carga_cols_n = df_to_sql_insert(df_cols_no_transformadas, "columnas_no_transformadas")
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    for i in range(len(df_cols_no_transformadas)):
        cursor.execute(sql_carga_cols_n, list(df_cols_no_transformadas.iloc[i]))
        conn.commit()
    cursor.close()
    conn.close()

def preparar_datos_no_cargados(df_datos, cols_n, id_inicial):
    datos_no_cargados = pd.DataFrame(df_datos[cols_n])
    datos_no_cargados.index = list(range(id_inicial, id_inicial + len(df_datos)))
    
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor(buffered=True)
    cols_n_id = {}
    sql_id_col_n = 'SELECT ID_columna_no_transformada FROM columnas_no_transformadas WHERE nombre = %s'
    for col in list(datos_no_cargados.columns):
        cursor.execute(sql_id_col_n, [col])
        id_col_n = cursor.fetchone()[0]
        cols_n_id[col] = id_col_n
    cursor.close()
    conn.close()
    datos_no_cargados = datos_no_cargados.rename(columns = cols_n_id)
    return datos_no_cargados

def cargar_datos_no_cargados(df_datos_no_cargados):
    datos_subir = pd.DataFrame({"valor":[],
                                "ID_columna_no_transformada":[],
                                "PATIENT_ID":[]})
    for i in range(len(df_datos_no_cargados)):
        for j in range(len(df_datos_no_cargados.iloc[0])):
            datos_subir = datos_subir.append({"valor": str(df_datos_no_cargados.iloc[i,j]),
                                              "ID_columna_no_transformada": df_datos_no_cargados.columns[j].tolist(),
                                              "PATIENT_ID": df_datos_no_cargados.index[i].tolist()}, ignore_index = True)
    sql_carga_datos_no_cargados = df_to_sql_insert(datos_subir, "datos_no_cargados")
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    for i in range(len(datos_subir)):
        cursor.execute(sql_carga_datos_no_cargados, list(datos_subir.iloc[i])) #Aqui esta el fallo
        conn.commit()
    cursor.close()
    conn.close()

In [52]:
def cargar(name): #Funcion final
    datos = pd.read_csv(name,  sep=';',encoding= 'unicode_escape') 
    (columnas_transformadas, columnas_no_transformadas) = dividir_columnas(datos)
    datos_finales = transformar_datos(datos, columnas_transformadas)
    id_inicial = cargar_datos_finales(datos_finales)
    id_carga = iniciar_carga(name)
    carga_columnas_transformadas(columnas_transformadas, id_carga)
    carga_columnas_no_transformadas(columnas_no_transformadas, id_carga)
    datos_no_cargados = preparar_datos_no_cargados(datos, columnas_no_transformadas, id_inicial)
    cargar_datos_no_cargados(datos_no_cargados)
    print("Datos cargados correctamente")

In [4]:
correspondencias = {
    "EDAD/AGE": "DMRAGEYR",
    "SEXO/SEX": "DMRGENDR",
    "F_INGRESO/ADMISSION_D_ING/INPAT": "DATAD",
    "UCI_DIAS/ICU_DAYS": "DATLGTI",
    "F_ALTA/DISCHARGE_DATE_ING": "DATDSI",
    "MOTIVO_ALTA/DESTINY_DISCHARGE_ING": "DSXOS",  
    #"F_INGRESO/ADMISSION_DATE_URG/EMERG": "", # No sale 
    #"HORA/TIME_ADMISION/ADMISSION_URG/EMERG": "",  # No sale 
    #"ESPECIALIDAD/DEPARTMENT_URG/EMERG": "",   # No sale 
    #"DIAG_URG/EMERG": "",   # No sale 
    #"DESTINO/DESTINY_URG/EMERG": "",  # No sale 
    "TEMP_PRIMERA/FIRST_URG/EMERG": "CSXBTPA",
    "FC/HR_PRIMERA/FIRST_URG/EMERG": "CSXCHRA",
    "GLU_PRIMERA/FIRST_URG/EMERG": "LBXSGLA",
    "SAT_02_PRIMERA/FIRST_URG/EMERG": "CSXOSTA",
    "TA_MAX_PRIMERA/FIRST/EMERG_URG": "CSXSYA",
    "TA_MIN_PRIMERA/FIRST_URG/EMERG": "CSXDIA",
    #"HORA/TIME_CONSTANT_ULTIMA/LAST_URG/EMERG": "",   # No sale
    "FC/HR_ULTIMA/LAST_URG/EMERG": "CSXCHRHn",
    "TEMP_ULTIMA/LAST_URG/EMERG": "CSXBTPHn",
    "GLU_ULTIMA/LAST_URG/EMERG": "LBXSGLHn",
    "SAT_02_ULTIMA/LAST_URG/EMERG": "CSXOSTHn",
    "TA_MAX_ULTIMA/LAST_URGEMERG": "CSXSYHn",
    "TA_MIN_ULTIMA/LAST_URG/EMERG": "CSXDIHn"
}
config = {'host': "localhost",
          'port': 3306,
          'user': "root",
          'passwd': "123456",
          'db': "datoscovid"}

In [55]:
def cargar_datos_no_cargados(df_datos_no_cargados):
    datos_subir = pd.DataFrame({"valor":[],
                                "ID_columna_no_transformada":[],
                                "PATIENT_ID":[]})
    for i in range(len(df_datos_no_cargados)):
        for j in range(len(df_datos_no_cargados.iloc[0])):
            datos_subir = datos_subir.append({"valor": str(df_datos_no_cargados.iloc[i,j]),
                                              "ID_columna_no_transformada": (df_datos_no_cargados.columns[j].tolist()),
                                              "PATIENT_ID": df_datos_no_cargados.index[i].tolist()}, ignore_index = True)
    return datos_subir
    
def cargar2(name): #Funcion final
    datos = pd.read_csv(name,  sep=';',encoding= 'unicode_escape') 
    (columnas_transformadas, columnas_no_transformadas) = dividir_columnas(datos)
    datos_finales = transformar_datos(datos, columnas_transformadas)
    id_inicial = cargar_datos_finales(datos_finales)
    id_carga = iniciar_carga(name)
    carga_columnas_transformadas(columnas_transformadas, id_carga)
    carga_columnas_no_transformadas(columnas_no_transformadas, id_carga)
    datos_no_cargados = preparar_datos_no_cargados(datos, columnas_no_transformadas, id_inicial)
    print(cargar_datos_no_cargados(datos_no_cargados))
    

In [58]:
cargar2('Datos_covid.csv')

                       valor  ID_columna_no_transformada  PATIENT_ID
0                          1                         1.0       681.0
1                 06/03/2020                         2.0       681.0
2                    4:02:00                         3.0       681.0
3      Medicina de Urgencias                         4.0       681.0
4    DIFICULTAD RESPIRATORIA                         5.0       681.0
..                       ...                         ...         ...
471                 19:00:00                         3.0       748.0
472    Medicina de Urgencias                         4.0       748.0
473  DIFICULTAD RESPIRATORIA                         5.0       748.0
474                  Ingreso                         6.0       748.0
475                 22:11:00                         7.0       748.0

[476 rows x 3 columns]


AttributeError: 'NoneType' object has no attribute 'head'

In [47]:
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

cursor.execute('DROP SCHEMA datoscovid;')

In [57]:
print(1==1.0)


True
